In [ ]:
%load_ext autoreload
%autoreload 2

import os
import time
from tqdm import tqdm_notebook as tqdm

import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers, losses, optimizers

import cv2

from core.layers import Downsampler, Upsampler, Skip
from core.losses import pixelwise_mse
from core.callbacks import SaveResultImage
from core.utils import crop_div_32

import imageio
import matplotlib.pyplot as plt

In [ ]:
INPUT_IMG_PATH = "data/lena.png"
OUTPUT_IMG_NAME = os.path.splitext(os.path.basename(INPUT_IMG_PATH))[0]
OUTPUT_DIR = "output/image_reconstruction"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

original_img = imageio.imread(INPUT_IMG_PATH)
original_img = crop_div_32(original_img)


mask = np.random.choice([1, 0], original_img.shape[:2], p=[0.5, 0.5]).astype(np.uint8)
img = cv2.bitwise_and(original_img, original_img, mask=mask)

img = tf.image.convert_image_dtype(img, dtype=tf.float32)
plt.imshow(img)

In [ ]:
num_filters_down = [128, 128, 128, 128, 128]
ksizes_down = [3, 3, 3, 3, 3]

num_filters_up = num_filters_down
ksizes_up = ksizes_down

num_filters_skip = [4, 4, 4, 4, 4]
ksizes_skip = [1, 1, 1, 1, 1]

sigma_p = 1/30
n_iter = 11000
lr = 0.001
upsampling_mode = "bilinear"

In [ ]:
skip_outputs = [None] * len(num_filters_skip)

model_input = layers.Input(shape=img.shape[:2] + (32,), dtype=tf.float32)

x = layers.GaussianNoise(sigma_p)(model_input) 

for i in range(len(num_filters_down)):
    x = Downsampler(num_filters_down[i],
                    ksizes_down[i])(x)
    if num_filters_skip[i]:
        skip_outputs[i] = Skip(num_filters_skip[i], ksizes_skip[i])(x)

    
for i in range(len(num_filters_up) - 1, -1, -1):
    if num_filters_skip[i]:
        x = tf.concat((x, skip_outputs[i]), axis=3)
    x = Upsampler(num_filters_up[i],
                  ksizes_up[i],
                  scale_factor=2,
                  upsampling_mode=upsampling_mode)(x)

    
# Transform to original 3-channel image
model_output = layers.Conv2D(filters=3, kernel_size=1, 
                             strides=1, padding='SAME', 
                             activation='sigmoid')(x)
model = models.Model(inputs=model_input, outputs=model_output)

model.summary()

In [ ]:
from core.callbacks import SaveResultImage

z = np.random.uniform(0, 0.1, img.shape[:2] + (32,))
z = np.expand_dims(z, axis=0)
y_true = np.expand_dims(img, axis=0)

callbacks = [
    SaveResultImage(n=200, input_tensor=z, output_dir=OUTPUT_DIR, img_name=OUTPUT_IMG_NAME)
]

# Extend the mask to 3 channels
mask = np.transpose(np.array([mask] * 3), [1, 2, 0])

model.compile(loss=pixelwise_mse(mask=mask),
              optimizer=optimizers.Adam(lr=lr))

model.fit(z, y_true, 
          epochs=n_iter,
          callbacks=callbacks)